In [1]:
import requests
import shopify
from bs4 import BeautifulSoup
import logging
from dotenv import load_dotenv
import os

logging.basicConfig(level=logging.ERROR)

shop_url = "https://429eef-90.myshopify.com/"
api_version = '2024-01'
# state = binascii.b2a_hex(os.urandom(15)).decode("utf-8")
# redirect_uri = "http://myapp.com/auth/shopify/callback"
# scopes = ['read_apps', 'write_files', 'read_files', 'write_products', 'read_products', 'read_content', 'write_content', 'write_product_feeds', 'read_product_feeds', 'write_product_listings', 'read_product_listings']

load_dotenv()
access_token = (os.getenv('access_token'))

session = shopify.Session(shop_url, api_version, access_token)
shopify.ShopifyResource.activate_session(session)


In [2]:

variant = 'luxury'
search_term = f"*{variant}*"
# variant_id
products = shopify.Product.find(query=search_term)

In [3]:
products

[product(8774006112586),
 product(8766892900682),
 product(8764450046282),
 product(8764450308426),
 product(8766889066826),
 product(8764450570570),
 product(8764450701642),
 product(8764465611082),
 product(8766898536778),
 product(8764465676618),
 product(8764465807690),
 product(8766894178634),
 product(8764465873226),
 product(8766894571850),
 product(8764466037066),
 product(8766894997834),
 product(8764466135370),
 product(8764466397514),
 product(8764466757962),
 product(8764466954570),
 product(8764466987338),
 product(8766889722186),
 product(8764467052874),
 product(8774042550602),
 product(8766868324682),
 product(8764467249482),
 product(8766879924554),
 product(8764467413322),
 product(8766893523274),
 product(8764467511626),
 product(8764467642698),
 product(8766868783434),
 product(8766876877130),
 product(8764467708234),
 product(8764467937610),
 product(8774014107978),
 product(8774014370122),
 product(8764468166986),
 product(8766879367498),
 product(8764468265290),


In [4]:
class Product:
    def __init__(self, name, spu, imgs, url, detail=None):
        self.name = name
        self.detail = """

        <div class="value"><p><span style="font-weight: 400;">Gel Polish is a new generation of nails care!</span></p>
<p><span style="font-weight: 400;">Forget about not lasting and peeling off colors, experience super shiny lasting and incredible quality of ReformA Gel Polishes.</span></p>
<p><span style="font-weight: 400;">Nine reasons thanks to which you will love them:</span></p>
<ol>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They have beautiful colors, you can find juicy neons as well as delicate nudes and all shades between them - a total of over 300 and we still introduce new ones!</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">Even a beginner could apply them easily - they have consistence of yoghurt so you can spread them comfortably.</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They have delicate smell.</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They cure fast so you save time and protect nails.</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They are self-levelling and have good adhesion to natural nails, gel, acrylic and acrylic gel.</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They are ideal for manicure and pedicure.</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They are incredibly lasting.</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They don't peel or flake off.</span></li>
<li aria-level="1" style="font-weight: 400;"><span style="font-weight: 400;">They guarantee beautiful covering after applying 1-2 layers.</span></li>
</ol>
<p><span style="font-weight: 400;"> </span></p>
<p><span style="font-weight: 400;">All gel polishes are available in bottles with brushes of capacity 10ml, and some of them also in bottles of capacity 3ml.</span></p>
<p><span style="font-weight: 400;">Try also our incredible bases and tops.</span></p>
<p><strong>ReformA Educator's advices:</strong></p>
<p><span style="font-weight: 400;">When you apply gel polish remember not to touch the underside with a brush.</span></p>
<p><span style="font-weight: 400;">Delicately stroking with a brush, apply gel polish in the air - then you will get smooth and beautiful surface.</span></p>
<p><span style="font-weight: 400;">Each color apply on cured base and at the end cover it with top.</span></p>
<p><span style="font-weight: 400;">Curing time in a UV lamp: 2 minutes, in an LED lamp: 1min - 20sec, in a 48W Dual LED lamp: 30 - 20sec.</span></p>
<p><span style="font-weight: 400;">The curing time of gel products and gel polishes depends on the power of lamp and bulbs, always check individually the curing time of gel or gel polish.</span></p>
"""
        if detail:
            self.detail = detail
        self.spu = spu
        self.imgs = imgs
        self.url = url
        self.price = '10.00'
        self.compare_at_price = '13.00'
        self.grams = 15
        self.weight = 0.015
        self.vendor = 'Reforma'
        self.product_type = 'Gel Polish'

    def __str__(self):
        return f"Product: {self.name}, SPU: {self.spu}"


In [5]:
def get_search_results(url):
    """
    Получает результаты поиска для заданного URL.
    """
    try:
        with requests.get(url) as response:
            if response.status_code == 200:
                return response.text
            else:
                logging.error("Ошибка при получении страницы с результатами поиска: %d", response.status_code)
                return None
    except requests.RequestException as e:
        logging.error("Ошибка при запросе страницы: %s", str(e))
        return None


In [43]:
def extract_product_links(url):
    """
    Извлекает ссылки на продукты из HTML-кода страницы с результатами поиска.
    """
    html = get_search_results(url)
    if html:
        soup = BeautifulSoup(html, 'html.parser')
        links = soup.find_all('h4')
        return [h3.find('a')['href'] for h3 in links if h3.find('a')]
    return None


In [7]:
def UkRefSearch(sku):
    """
    Извлекает ссылки на продукты из HTML-кода страницы с результатами поиска.
    """
    url = 'https://reformacosmetics.com/search/?q='+str(sku)+'&s=%D0%9F%D0%BE%D1%88%D1%83%D0%BA'

    html = get_search_results(url)
    if html:
        soup = BeautifulSoup(html, 'html.parser')
        links = soup.find_all('a', class_='nm')
        return [link['href'] for link in links]
    return None


In [8]:
shopify.Variant.to_dict


<function pyactiveresource.activeresource.ActiveResource.to_dict(self)>

In [9]:
def letsgo(page):
    print()

    # url = 'https://reforma.top/catalogsearch/result/?q=' + SPU + '&product_list_limit=144'
    url = 'https://reforma.top/shop/nails/gel-polish/base-tops/?product_list_limit=144&p='+str(page)
    print(url)
    product_links = extract_product_links(url)
    if product_links:
        print("List created")
        return product_links
        

In [30]:
def extract_data_from_url(url):
    """
    Extracts product data from the provided URL.
    """
    try:
        # Extract data from the URL
        html = get_search_results(url)
        if not html:
            raise ValueError("Failed to fetch HTML content from the URL.")

        soup = BeautifulSoup(html, 'html.parser')
        for a_tag in soup.find_all('a', href=True):
            del a_tag['href']        
        description_div = soup.find('div', id='tab-description')

        # Remove h3 and li elements with data attribute 'data-number-of-phrases="9"'
        if description_div:
            for element in description_div.find_all(['h3', 'li']):
                if 'data-number-of-phrases="9"' in str(element):
                    element.extract()

        # Convert the remaining div element to a string
        detail = str(description_div) if description_div else ""
        spu = soup.find('span', class_='sku').text
        name = soup.find('h1',class_='product_title entry-title').text
        imgs = [img.get('data-large_image') for img in soup.find_all('img',decoding="async")]
        # imgs =soup
        
        return Product(name, spu, imgs, url, detail=detail)
    except Exception as e:
        logging.error(f"Failed to extract data from URL: {str(e)}")
        return None

In [11]:
def create_product(product):
    """
    Creates a new product on Shopify with the provided data.
    """
    try:
        Handle = product.url.split('/')[-1].replace('.html','')
        product_shopify = shopify.Product.find(handle=Handle)
        print(product.name)
        Handle = product.name.rstrip().replace(" - ", " ").replace(" ", "-").replace(",", "").casefold()
        
        if not product_shopify:
            # Create a new product instance
            product_shopify = shopify.Product()

            # Set basic product details
            product_shopify.title = product.name
            product_shopify.handle = Handle
            product_shopify.body_html = product.detail
            product_shopify.vendor = product.vendor
            product_shopify.product_type = product.product_type

            # Create a default variant
            default_variant = {
                'title': 'Default Title',
                'price': product.price,
                'sku': product.spu,
                'compare_at_price': product.compare_at_price,
                'inventory_policy': 'deny',
                'fulfillment_service': 'manual',
                'inventory_management': 'shopify',
                'option1': 'Default Title',
                'taxable': True,
                'grams': product.grams,
                'weight': product.weight,
                'weight_unit': 'kg',
                'requires_shipping': True
            }

            # Create a Variant object for the default variant
            variant = shopify.Variant(default_variant)

            # Add the default variant to the product
            product_shopify.variants = [variant]

            # Add images to the product
            half_len = len(product.imgs) // 2
            images = [{'src': img_url, 'position': index, 'alt': product.name} for index, img_url in enumerate(product.imgs[:half_len], start=1)]
            product_shopify.images = images
            logging.info("Product created successfully!")
            print("Product created successfully!")
            product_shopify.save()

        elif product_shopify[0].handle != Handle:
            # Update the product title
            product_shopify = product_shopify[0]
            print()
            print(product_shopify.handle)
            print(Handle)
            print()
            product_shopify.handle = Handle

            product_shopify.save()

            print("Product mody successfully!")

        else:
            # Update the product title
            # product_shopify = product_shopify[0]
            

            # product_shopify.body_html = product.detail
            # product_shopify.vendor = product.vendor
            # product_shopify.product_type = product.product_type
            logging.info("Product modified successfully!")
            print("Product already successfully!")
        
        # Save the product
        # product_shopify.save()

        return product_shopify
    except Exception as e:
        logging.error(f"Failed to create product: {str(e)}")
        return None

In [12]:
# prod_links = letsgo(1)
# for index, prod_link in enumerate(prod_links, start=1):
#     print(prod_link)
#     create_product(extract_data_from_url(prod_link))
#     print(f"{index}/{len(prod_links)}\n")

In [41]:
extract_product_links('https://komilfo.ua/en/product-category/komilfo-gel-polish/')

['https://komilfo.ua/en/product-category/komilfo-gel-polish/stone-collection-en/',
 'https://komilfo.ua/en/product-category/komilfo-gel-polish/komilfo-liquid-glam-gel-en/',
 'https://komilfo.ua/en/product-category/komilfo-gel-polish/stardust-glitter-en/',
 'https://komilfo.ua/en/product-category/komilfo-gel-polish/kaleidoscopic-collection-en/',
 'https://komilfo.ua/en/product-category/komilfo-gel-polish/macarons-collection/',
 'https://komilfo.ua/en/product-category/komilfo-gel-polish/rior-collection/',
 'https://komilfo.ua/en/product-category/komilfo-gel-polish/luminous-collection-en/',
 'https://komilfo.ua/en/product-category/komilfo-gel-polish/holographic-collection/',
 'https://komilfo.ua/en/product-category/komilfo-gel-polish/dragon-fruit-collection/',
 'https://komilfo.ua/en/product-category/komilfo-gel-polish/moon-crush-collection/',
 'https://komilfo.ua/en/product-category/komilfo-gel-polish/glow-attack-collection/',
 'https://komilfo.ua/en/product-category/komilfo-gel-polish/d

In [44]:
extract_product_links('https://komilfo.ua/ru/product-category/komilfo-gel-polish-ru/french-collection/')

['https://komilfo.ua/ru/product/gel-lak-komilfo-french-collection-f001-blednyj-lilovo-rozovyj-emal-dlya-frencha-15-ml/',
 'https://komilfo.ua/ru/product/gel-lak-komilfo-deluxe-series-d200-svetlyj-rozovo-oranzhevyj-emal-dlya-frencha-8-ml/',
 'https://komilfo.ua/ru/product/gel-lak-komilfo-french-collection-f002-nezhno-rozovyj-emal-dlya-frencha-15-ml/',
 'https://komilfo.ua/ru/product/gel-lak-komilfo-french-collection-f002-nezhno-rozovyj-emal-dlya-frencha-8-ml/',
 'https://komilfo.ua/ru/product/gel-lak-komilfo-french-collection-f003-blednyj-persikovo-rozovyj-emal-dlya-frencha-15-ml/',
 'https://komilfo.ua/ru/product/gel-lak-komilfo-french-collection-f003-blednyj-persikovo-rozovyj-emal-dlya-frencha-8-ml/',
 'https://komilfo.ua/ru/product/gel-lak-komilfo-french-collection-f004-zhemchuzhno-rozovyj-emal-dlya-frencha-15-ml/',
 'https://komilfo.ua/ru/product/gel-lak-komilfo-french-collection-f004-zhemchuzhno-rozovyj-emal-dlya-frencha-8-ml/',
 'https://komilfo.ua/ru/product/gel-lak-komilfo-frenc

In [31]:
data = extract_data_from_url('https://komilfo.ua/en/product/gel-polish-komilfo-stone-collection-st002-dark-gray-with-crumbs-8-ml/')
print(data.detail)


<div aria-labelledby="tab-title-description" class="woocommerce-Tabs-panel woocommerce-Tabs-panel--description panel entry-content wc-tab" id="tab-description" role="tabpanel">
<h2>Description</h2>
<h2><span class="VIiyi" lang="en"><span class="JLqJ4b ChMk0b" data-language-for-alternatives="en" data-language-to-translate-into="ru" data-number-of-phrases="1" data-phrase-index="0">The advantage of choosing Komilfo products:</span></span></h2>
<ul>
<li><strong>The new series of the updated Komilfo palette</strong> is high-quality gel polishes at an affordable price, developed by masters for masters!</li>
<li>Soft, stylish shades, with small inclusions, are impressive and resemble an expensive and exquisite work of stone.</li>
<li>The colors lay down softly, do not flow and do not require much effort in application, the particles do not bulge</li>
</ul>
<h3 class="text-dost-title">TECHNOLOGY FOR APPLYING KOMILFO GEL POLISHES:</h3>
<ol>
<li>Do the standard nail preparation: peel off the pre